In [231]:
import pandas as pd
import numpy as np
import axelrod as axl
import pathlib 

In [187]:
df = pd.read_csv("outcomes.csv", header=None)
df.columns = ["strategy 1", "strategy 2", "score 1", "score 2", "count"]

In [188]:
df.describe()

,score 1,score 2,count
count,794731.000000,794731.000000,794731.000000
mean,2.210296,2.274771,5.546594
std,0.749463,0.879763,41.762043
min,0.000000,0.000000,1.000000
25%,1.720000,1.595000,1.000000
50%,2.180000,2.320000,1.000000
75%,2.665000,2.920000,3.000000
max,5.000000,5.000000,1000.000000


Creating a dictionary `f` that maps a name of strategy tuples to the mean utility for that matchup:


In [189]:
utilities = {pair: (f["score 1"].mean(), f["score 2"].mean()) 
             for pair, f in df.groupby(["strategy 1", "strategy 2"])}

In [209]:
def scores(strategy_pair, utilities=utilities):
    """
    Return the fitness scores:
    
        [a, b]
        [c, d]
    
    for a pair of strategies, in a population of N individuals 
    with i individuals of the first type.
    """
    try:
        s1, s2 = strategy_pair
        scores = np.array([[utilities[(s1, s1)][0], utilities[(s1, s2)][0]], 
                           [utilities[(s1, s2)][1], utilities[(s2, s2)][0]]])
    except KeyError:
        s2, s1 = strategy_pair
        scores = np.array([[utilities[(s2, s2)][0], utilities[(s1, s2)][1]], 
                           [utilities[(s1, s2)][0], utilities[(s1, s1)][0]]])
    return scores

In [212]:
scores(("Alternator", "Tit For Tat"))

array([[ 2.   ,  2.515],
       [ 2.49 ,  3.   ]])

In [213]:
def fitness(strategy_pair, N, i, utilities=utilities):
    """
    Return the fitness of a strategy pair in a population with 
    N total individuals and i individuals of the first type.
    """
    m = scores(strategy_pair, utilities)    
    return np.dot(m, np.array([i, N - i]))

In [214]:
# Compute fitness for case of Alternator invading population of Cooperators
fitness(("Alternator", "Tit For Tat"), 5, 1)

array([ 12.06,  14.49])

In [215]:
def transition(strategy_pair, N, i, utilities=utilities):
    """
    Return the 3 transition probabilities:
    
    P[i, i - 1]
    P[i, i]
    P[i, i + 1]
    
    Assuming: 
     - a given stratgy pair, 
     - a given total population size N
     - and a state i (the number of individuals of the first type)
    """
    fit = fitness(strategy_pair, N, i, utilities=utilities)
    p_up = fit[0] * i / (fit[0] * i + fit[1] * (N - i)) * (N - i) / N
    p_down = fit[1] * (N - i) / (fit[0] * i + fit[1] * (N - i)) * (N - i) / N
    p_stay = 1 - p_up - p_down
    return p_down, p_stay, p_up

In [218]:
# Compute transitions probabilitis for the case of Alternator invading population of Cooperators
transition(("Alternator", "Tit For Tat"), 5, 1)

(0.66221079691516715, 0.19999999999999996, 0.13778920308483292)

In [219]:
# Using Marc's code with a modification 
# (just so that it fits with what I've 
# done so far but not precious about this)

def fixation(strategy_pair, N, i=1, utilities=utilities):
    """Return the fixation probability for each pair"""
    ratios = []
    for j in range(1, N): # ignore first and last transitions, which are likely zero
        p_down, _, p_up = transition(strategy_pair, N, j, utilities=utilities)
        ratios.append(p_down / p_up)
    t = np.cumprod(ratios)
    s = np.cumsum(t)
    if i > 1:
        return (1 + s[i - 2]) / (1 + s[-1])
    return 1 / (1 + s[-1])

In [225]:
# Confirm that the calculation works with i != 1.
fixation(("Alternator", "Tit For Tat"), 5), 1 - fixation(("Tit For Tat", "Alternator"), 5, i=4)

(0.041959889586422321, 0.041959889586422272)

In [249]:
def write_fixation(N):
    """
    Write fixation probabilities for all 
    strategy pairs to a file called 'results/fixation_N.csv'
    
    File looks like:
    
    s1, s2, x_i, x_{N - 1}
    """
    path = pathlib.Path("results")
    path.mkdir(exist_ok=True)
    path = pathlib.Path("results") / "fixation_{}.csv".format(N)

    with path.open("w") as textfile:
        for strategy_pair in utilities:
            if strategy_pair[0] != strategy_pair[1]:
                fixation_probabilities = fixation(strategy_pair, N), fixation(strategy_pair, N, N - 1)
                textfile.write("{}, {}, {}, {}\n".format(*strategy_pair, *fixation_probabilities))

In [250]:
# Write all of these to file
for N in range(2, 12 + 1):
    write_fixation(N)